# Linear Regression in Python

Linear regression in Python is available through the *sklearn* machine learning library. In this hands-on exercise we will examine the relationship between property prices and poverty levels in a particular Boston neighborhood. To begin let's import some libraries:

In [ ]:
import numpy as np
import pandas as pd
import scipy.stats as stats
import sklearn

The numpy library provides us with some convenient ways to represent and manipulate vectors and matrices, which we will use extensively in this course. 

The pandas library provides very nice tools for managing datasets, while scipy is the Python scientific library with useful functions. 

The main star of this show is sklearn, which is the Python machine learning library, providing us with tools to create regression models using linear regression, and classification models using many models including Bayesian models and Support Vector Machines, which we will look at in this lecture.

In the lab we will see how to use our own datasets, but for this lecture we will use some toy datasets that are given in sklearn. The toy datasets, amongst many others, include:

- boston: Housing prices against factors like crime rate, plot ratio, poverty levels, etc, great for playing with regression models.
- iris: Dataset on iris flowers for classification.
- diabetes: Another regression dataset.
- digits: A collection of 8x8 images of digits for classification.
- wines: Data on wine types for classification.
- breast cancer: Breast cancer dataset for regression.

## Loading the Boston Dataset
https://scikit-learn.org/stable/datasets/index.html

For now we will use the Boston dataset. To load it:


In [ ]:
import pandas as pd
import numpy as np

data_url = "http://lib.stat.cmu.edu/datasets/boston"
raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
target = raw_df.values[1::2, 2]
feature_names = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 
 'B', 'LSTAT']


We can see how big out dataset is:

In [ ]:
data.shape

From here we see that there's 506 pieces of data, with 13 attributes each. Let's look at what's in this dataset by printing the first 5 entries:

In [ ]:
np.set_printoptions(precision = 3, suppress = True)
d = repr(data[:5]) # Nice printable version of boston.data
print(d)

As we can see we have a Numpy array of 13 attributes, just as we found when we looked at the shape of the array. 

This gives us a very nice description of the 13 attributes that are available for regression. There is a 14th attribute which is MEDV, the median price value of properties in thousands of dollars.

## Creating a Pandas Dataframe
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html

To make it easier to manipulate the data, let's convert it to a Pandas dataframe, and print the first five rows:

In [ ]:
bos = pd.DataFrame(data)
bos.head()

Very nice, but the columns don't have headings. This is set using the list bos.columns, and as it turns out the headings are available as a list in boston.feature_names. We can just set the bos.columns accordingly.

In [ ]:
bos.columns = feature_names
print(feature_names)
bos.head()

We can now explore correlations between various columns of our data. For example, we may expect crime rate to be higher in industrial areas than in residential areas, that poverty (lstat - % of population in 'lower status') is correlated with age, or that crime rates are inversely proportional to more valuable properties, reflected by higher tax rates. We make use of correlation to test these ideas. 

A correlation of 1 between two factors A and B means that factor A depends perfectly on factor B, while a correlation of -1 means that factor A is perfectly negatively correlated with factor B. We can find correlation using:

```
dataframe['A'].corr(dataframe['B'])
```


In [ ]:
crime_indus = bos['CRIM'].corr(bos['INDUS'])
pov_age = bos['LSTAT'].corr(bos['AGE'])
crime_tax = bos['CRIM'].corr(bos['TAX'])

print("corr crime/industry: %3.3f, corr poverty/age: %3.3f, corr crime/tax: %3.3f" 
      % (crime_indus, pov_age, crime_tax))


We see that there is some correlation between crime rate and industrial areas, and as expected poverty is strongly correlated with age. What is less expected is that there is a strong positive correlation between crime and tax rate (!!). This is something worth looking at. :) But that's not why we are here.

## Finding the Relationship Between Property Prices and Poverty Levels

Let's see how we can create a linear regression model to find the relationship between property prices and poverty levels. In the Boston dataset, we can find the property prices in boston.target. We first create a new column called "price" in the Pandas dataframe, and then check that there is actually a correlation between property prices and poverty levels.

In [ ]:
bos['PRICE'] = target
print("Correlation between property prices and poverty levels: %3.3f"
     % bos['PRICE'].corr(bos['LSTAT']))

Here we can see a very sharp negative correlation between property prices and poverty levels, as we would expect. Let's start building our training data from the Pandas dataframe:

### Creating the Data

We will create the training data by extracting the target or "dependent variable" (property prices) and the data driving the target (the "independent variable"). We then use linear regression to find the model relating independent and dependent variables.  We begin by converting both variables to from row vectors to column vectors:

In [ ]:
X = bos['LSTAT'].values.reshape(-1, 1)
Y = bos['PRICE'].values.reshape(-1, 1)

### Splitting into Training and Testing Data
https://scikit-learn.org/stable/model_selection.html

Before we create a model we always want to split the data into training data and testing (or validation) data. This allows us to test the model's "goodness of fit" against data it has seen for training, and data it has never seen before ('unseen data'). This is to test for "overfitting", where the model memorizes the training data and has excellent result, but produces very poor results for unseen data. We want to ensure that the results for both training data and testing data do not vary too greatly.

We will use the train_test_split function in sklearn to put aside 33% of the data for testing. 

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.33, random_state=0)


We will verify that we indeed have 33% of our data for testing:

In [ ]:
all_rows = X.shape[0]
test_rows = X_test.shape[0]
print("%%age of data used for test: %3.2f%%" % (test_rows / all_rows * 100.0))

### Creating the Regression Model

Excellent! Now let's build our regression model. We start by importing the LinearRegression class from sklearn:

In [ ]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()

Now we can use the fit method to learn from our data, and look at the coefficient and intercept for our model:


In [ ]:
lm.fit(X_train, Y_train)
print("Coefficient: %3.4f, Intercept: %3.4f." % (lm.coef_, lm.intercept_))

The coefficient tells us how property prices fall (since it is negative) for each percent increase in poverty, while the intercept is a base value at 0% poverty.  This gives us some interesting insight into how poverty affects property prices in Boston.

### Evaluating for Over-fitting

We briefly mentioned overfitting earlier in this document; a model has "overfitted" its training data if it can perform very well predicting the outputs for training data, yet perform very poorly on data it has never seen before.

So now lets now look at how well this model fits the training data and the testing data. We will take the root-mean-square error (RMSE) given by:

$$
RMSE = \sqrt{\frac{\sum_{i=0}^{n-1}(y_{a,i} - y_{m,i})^2}{n} }
$$

Here $y_{a,i}$ is the i-th *actual* value from the data, and $y_{m, i}$ is the corresponding output from the model.

On its own the RMSE is fairly useless, but we can use it to compare the prediction error of the model when it is using training data it has seen, and testing data it has never seen.

We call lm.predict(X_train) and lm.predict(X_test) to produce the predicted property prices using the training and testing data respectively, then call the mean_squared_error function from the sklearn.metrics package and square-root the result. Recall that our target values are in Y_train and Y_test:

In [ ]:
from sklearn import metrics
Y_pred_train = lm.predict(X_train)
Y_pred_test = lm.predict(X_test)
train_mse = np.sqrt(metrics.mean_squared_error(Y_train, Y_pred_train))
test_mse = np.sqrt(metrics.mean_squared_error(Y_test, Y_pred_test))
print("RMSE for training data: %3.4f. RMSE for testing data: %3.4f." % (train_mse, test_mse))
                

On its own the RMSE is fairly close. This means that the model performs about as well on data it has never seen, as on training data it has seen. So we are confident that the model is sufficiently "general" and has not memorized the training data to the detriment of unseen data.

Note however that the RMSE does not give us a really good idea of how accurate our model is in absolute terms, just in relative terms. We can still use this however to compare against other models we create.



### Playing with Our Model

We can now make some predictions on our model. We look at property prices if the poverty level was 10%, versus prices when the property level is 25%. We note that the prices are in units of $1,000 and multiply accordingly.

Note that lm.predict requires a 2D numpy array, which we create using np.array.

In [ ]:
price1 = lm.predict(np.array([[10.0]]))
price2 = lm.predict(np.array([[25.0]]))

print("Price at 10%% poverty level is $%3.2f. Price at 25%% poverty level is $%3.2f"
      %(price1 * 1000, price2 * 1000))


From here we see that at a 10% poverty level the mean pricing for housing is \\$25,129.90, which drops to \\$10,868.97 when the poverty level rises to 25%. We can manually verify that this is true:

In [ ]:
p1 = -0.9507 * 10 + 34.6372
p2 = -0.9507 * 25 + 34.6372

print("Price at 10%% poverty level is $%3.2f. Price at 25%% poverty level is $%3.2f"
      %(p1 * 1000, p2 * 1000))

We see an almost perfect match between the two models. :)